# Tests módulo `ariaclinprot`

In [1]:
import ariaclinprot as acp

In [ ]:
acp.convertPrescriptionToClinicalProtocol(prescription='../../prescripciones/SBRTPulmon7.5x8.csv', ProtocolID='SBRTPulmon7.5x8', 
                                          TreatmentSite='Lung', PlanID='SBRTPulmon')

## Prescripciones

In [5]:
%ls ../..

LICENSE*  prescripciones/  protocolos/  README.md*  tools/


In [2]:
pvdf, ccdf, oardf = acp.parse_prescription('../../prescripciones/SBRTPulmon7.5x8.csv')

In [3]:
pvdf

,Volume,Dose,FxDose
0,PTV LSI,60.000,7.500


In [4]:
ccdf

,Volume,Min,Max,AtLeast,NoMore
0,PTV LSI,,,"[95, 95, 57]","[5, 107, 64.2]"


In [5]:
oardf.Dmean = oardf.Dmean.str.strip()
oardf.Dmax = oardf.Dmax.str.strip()
oardf

,Organ,Dmean,Dmax,DosimPars
0,suma pulmones,,,"[D 950cc$14.4 Gy, V15.2$37%]"
1,medula,,,[V32$1 cc]
2,plexo braquial,,33.6 Gy,[V26.4$0.35 cc]
3,via aerea,,48.8 Gy,[V 38.4$0.5 cc]
4,traquea,,56 Gy,[V50$4 cc]
5,esofago,,43.2 Gy,[V36.8$5cc]
6,corazon,,40 Gy,[V34.4$15 cc]
7,aorta,,62 Gy,[V55.2$10cc]
8,v cava inferior,,62 Gy,[V55.2$10cc]
9,v cava superior,,62 Gy,[V55.2$10cc]


In [11]:
oardf.DosimPars.values[1][1]

'V60$30 %'

In [15]:
acp.parseDosimPar(oardf.DosimPars.values[6][0])

{'Vxx': {'Volume%': 30.0, 'DoseGy': 60.0}}

## Escritura de protocolos clínicos

In [14]:
# Read protocol template
bbx = acp.parseProt('BareBone.xml')
# Preview
ProtocolID = 'Laringe'
TreatmentSite = 'Head and Neck'
acp.modPreview(bbx, ID=ProtocolID, TreatmentSite=TreatmentSite)
# Phases
PlanID='ORL'
FractionCount = int(float(pvdf.Dose[0])/float(pvdf.FxDose[0]))
acp.modPhase(bbx, ID=PlanID, vFractionCount=FractionCount)
# Structutures
for pv in pvdf.itertuples():
    acp.addStructure(bbx, structureName=pv.Volume)
for oar in oardf.itertuples():
    acp.addStructure(bbx, structureName=oar.Organ)

# Plan objetives
for pv in pvdf.itertuples():
    ccVolumedf = ccdf[ccdf.Volume == pv.Volume]
    '''
    if not ccVolumedf.Min.str.match(' +')[0]:
        a=1
    if ccVolumedf.Max.str.match(' +')[0] == False:
        a=1
    '''
    if ccVolumedf.AtLeast.values[0]:
        atLeastlst = ccVolumedf.AtLeast.values[0]
        VolumePercentage =  atLeastlst[0]
        DosePercentage = float(atLeastlst[1])/100
        FxDoseGy = float(pv.FxDose) * DosePercentage
        DoseGy = float(pv.Dose) * DosePercentage
        acp.addPlanObjetive(bbx, ID=pv.Volume, vParameter=VolumePercentage,
                            vDose=FxDoseGy, vTotalDose=DoseGy, vModifier=0)
    if ccVolumedf.NoMore.values[0]:
        noMorelst = ccVolumedf.NoMore.values[0]
        VolumePercentage =  noMorelst[0]
        DosePercentage = float(noMorelst[1])/100
        FxDoseGy = float(pv.FxDose) * DosePercentage
        DoseGy = float(pv.Dose) * DosePercentage
        acp.addPlanObjetive(bbx, ID=pv.Volume, vParameter=VolumePercentage,
                            vDose=FxDoseGy, vTotalDose=DoseGy)
for oar in oardf.itertuples():
    if oar.Dmean:
        ID = oar.Organ
        Parameter = 0
        Fxs = float(pvdf.Dose.values[0]) / float(pvdf.FxDose.values[0])
        TotalDose = acp.parseDose(oar.Dmean)
        Dose = f'{TotalDose / Fxs:.5f}'
        acp.addPlanObjetive(bbx, ID=ID, vParameter=Parameter, vDose=Dose, vTotalDose=TotalDose,
                            vModifier=8)
    if oar.Dmax:
        ID = oar.Organ
        Parameter = 0
        Fxs = float(pvdf.Dose.values[0]) / float(pvdf.FxDose.values[0])
        TotalDose = acp.parseDose(oar.Dmax)
        Dose = f'{TotalDose / Fxs:.5f}'
        acp.addPlanObjetive(bbx, ID=ID, vParameter=Parameter, vDose=Dose, vTotalDose=TotalDose,
                            vModifier=10)
    if oar.DosimPars:
        ID = oar.Organ
        for DosimPar in oar.DosimPars:
            constraint_dict = acp.parseDosimPar(DosimPar)
            for key, constraint in constraint_dict.items():
                if key == 'Vxx':
                    VolumePercentage = constraint['Volume%']
                    Fxs = float(pvdf.Dose.values[0]) / float(pvdf.FxDose.values[0])
                    TotalDose = constraint['DoseGy']
                    Dose = f'{TotalDose / Fxs:.5f}'
                    acp.addPlanObjetive(bbx, ID=ID, vParameter=VolumePercentage, vDose=Dose, vTotalDose=TotalDose,
                                        vModifier=1)
                    
# Quality Indexes
for pv in pvdf.itertuples():
    ccVolumedf = ccdf[ccdf.Volume == pv.Volume]
    if ccVolumedf.AtLeast.values[0]:
        TreatmentDosePrescription = acp.getTreatmentDosePrescription(pvdf)
        atLeastlst = ccVolumedf.AtLeast.values[0]
        VolumePercentage =  atLeastlst[0]
        DosePercentage = float(atLeastlst[1])/100
        StructureRelativeDose = float(pv.Dose) * DosePercentage / TreatmentDosePrescription * 100
        acp.addQualityIndex(bbx, ID=pv.Volume, vType=2, vModifier=0, 
                            vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                            vReportDQPValueInAbsoluteUnits='false')
    if ccVolumedf.NoMore.values[0]:
        TreatmentDosePrescription = acp.getTreatmentDosePrescription(pvdf)
        noMorelst = ccVolumedf.NoMore.values[0]
        VolumePercentage = noMorelst[0]
        DosePercentage = float(noMorelst[1])/100
        StructureRelativeDose = float(pv.Dose) * DosePercentage / TreatmentDosePrescription * 100
        acp.addQualityIndex(bbx, ID=pv.Volume, vType=2, vModifier=1, 
                            vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                            vReportDQPValueInAbsoluteUnits='false')

for oar in oardf.itertuples():
    if oar.DosimPars:
        ID = oar.Organ
        for DosimPar in oar.DosimPars:
            constraint_dict = acp.parseDosimPar(DosimPar)
            for key, constraint in constraint_dict.items():
                if key == 'Vxx':
                    VolumePercentage = constraint['Volume%']
                    PrescriptionDoseGy = pvdf.Dose.astype('float').max()
                    ConstraintDoseGy = constraint['DoseGy']
                    StructureRelativeDose = f'{ConstraintDoseGy / PrescriptionDoseGy * 100:.5f}'
                    acp.addQualityIndex(bbx, ID=ID, vType=2, vModifier=1, 
                                        vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                                        vReportDQPValueInAbsoluteUnits='false')
                if key == 'Dxx':
                    VolumePercentage = constraint['Volume%']
                    StructureRelativeDose = constraint['Dose%']
                    acp.addQualityIndex(bbx, ID=ID, vType=4, vModifier=1, 
                                        vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                                        vReportDQPValueInAbsoluteUnits='false')

# Write clincial protocol
acp.writeProt(bbx)

In [31]:
pvdf.Dose.astype('float').max()

63.0

In [5]:
bbx = acp.parseProt('BareBone.xml')
TreatmentSite = 'Prostata + vvss hipofx'
acp.modPreview(bbx, ID='ArmaduraTest', TreatmentSite=TreatmentSite)
acp.modPhaseID(bbx, 'Bach')
structureName = 'Recto'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, ID=structureName, vParameter=85, vDose=1, vTotalDose=20, vModifier=11)
structureName = 'PTV'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, structureName, vParameter=95, vDose=3*.95, vTotalDose=60*.95, vModifier=0, vPrimary='true')
acp.addPlanObjetive(bbx, structureName, vParameter=5, vDose=3*1.07, vTotalDose=60*1.07)
structureName = 'Recto'
acp.addQualityIndex(bbx, ID= structureName, vType=5, vModifier=1, vValue=85, vTypeSpecifier=20, vReportDQPValueInAbsoluteUnits='true')
acp.writeProt(bbx, protout='TestSalida.xml')

In [10]:
import xml.etree.ElementTree as ET

In [11]:
px = ET.parse('Próstata 20Fx.xml')
ET.indent(px)
px.write('Próstata 20Fx.xml', encoding='utf-8', xml_declaration=True)

### Diccionario de estructuras

In [19]:
import pandas as pd

In [31]:
StructureNormalizationdf= pd.DataFrame([
    {'Name' : 'Parotida_D', 'OtherNames' : 'parótida derecha'},
    {'Name' : 'Parotida_I', 'OtherNames' : 'parótida izda'},
    {'Name' : 'Medula', 'OtherNames' : 'médula'},
    {'Name' : 'Medula', 'OtherNames' : 'médula'},
    {'Name' : 'Gland_lagrimal_D', 'OtherNames' : 'glandula lacrimal derecha, glandula lacrimal der, glandula lacrimal dcha, gland lacrimal derecha, gland lacrimal der, gland lacrimal dcha'},
    {'Name' : 'Gland_lagrimal_I', 'OtherNames' : 'glandula lacrimal izquierda, glandula lacrimal izq, glandula lacrimal izqda, gland lacrimal izquierda, gland lacrimal izq, gland lacrimal izqda'},
])
StructureNormalizationdf

,Name,OtherNames
0,Gland_lagrimal_D,"glandula lacrimal derecha, glandula lacrimal d..."
1,Gland_lagrimal_I,"glandula lacrimal izquierda, glandula lacrimal..."


In [32]:
StructureNormalizationdf.OtherNames.str.contains('glandula lacrimal derecha')

0     True
1    False
Name: OtherNames, dtype: bool

In [35]:
StructureNormalizationdf[StructureNormalizationdf.OtherNames.str.contains('glandula lacrimal derecha')].Name.values[0]

'Gland_lagrimal_D'